In [1]:
import time
import os
import requests
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from PIL import Image
from io import BytesIO

# 웹드라이버 실행
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# 크롤링할 URL 설정
url = "https://www.airbnb.co.kr/s/%EC%84%9C%EC%9A%B8/homes?refinement_paths%5B%5D=%2Fhomes"

# Airbnb 사이트 열기
driver.get(url)

# 페이지 로딩 대기
time.sleep(5)

print("✅ Airbnb 페이지가 성공적으로 열렸습니다.")

# 이미지 저장 폴더 생성
img_folder = "airbnb_images"
os.makedirs(img_folder, exist_ok=True)

# 데이터 저장 리스트
data_list = []

# 함수: 숙소 데이터를 가져오는 부분
def get_data_from_page():
    global data_list

    # 숙소 목록에서 링크 가져오기
    for i in range(1, 19):  # 상위 18개 숙소 확인
        try:
            # 숙소 링크 요소 찾기
            xpath = f"(//div[contains(@class, 'c4mnd7m')])[{i}]//a"
            link_element = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, xpath)))

            # 숙소 URL 가져오기
            link = link_element.get_attribute("href")
            print(f"🔹 {i}. 숙소 링크: {link}")

            # 새 탭 열기
            driver.execute_script(f"window.open('{link}', '_blank');")
            time.sleep(3)

            # 모든 열린 탭 가져오기
            all_windows = driver.window_handles
            driver.switch_to.window(all_windows[-1])  # 새로 열린 탭으로 이동

            # 첫 번째 이미지 요소 찾기
            img_url = None
            try:
                img_element = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, "//picture//img"))
                )
                img_url = img_element.get_attribute("src")

                if img_url:
                    print(f"   🖼️ 첫 번째 이미지 URL: {img_url}")

                    # 이미지 다운로드 및 저장
                    response = requests.get(img_url)
                    if response.status_code == 200:
                        img = Image.open(BytesIO(response.content))
                        img_path = os.path.join(img_folder, f"숙소_{i}.jpg")
                        img.save(img_path)
                        print(f"   ✅ 이미지 저장 완료: {img_path}")
                    else:
                        print(f"   ❌ 이미지 다운로드 실패")
                else:
                    print("   ❌ 이미지 URL을 찾을 수 없습니다.")

            except Exception as e:
                print(f"   ❌ 첫 번째 이미지를 가져오는 데 실패했습니다: {e}")

            # 숙소 평점(리뷰 점수) 가져오기
            rating = None
            try:
                # 기존 XPath로 평점 추출 시도
                rating = WebDriverWait(driver, 3).until(
                    EC.presence_of_element_located((By.XPATH, "//div[@class='r1lutz1s atm_c8_o7aogt atm_c8_l52nlx__oggzyc dir dir-ltr']"))
                ).text.strip()

                # 평점 문자열에서 숫자만 추출 (숫자와 점만 남기기)
                rating = ''.join([c for c in rating if c.isdigit() or c == '.'])

                # 문자열을 숫자(실수)로 변환
                rating = float(rating) if rating else None
            except Exception as e:
                print(f"   ❌ 기존 XPath로 평점 추출 실패: {e}")

            # 새로운 XPath로 평점 추출 시도
            if rating is None:
                try:
                    rating = WebDriverWait(driver, 3).until(
                        EC.presence_of_element_located((By.XPATH, "//*[@id='site-content']/div/div[1]/div[3]/div/div[1]/div/div[2]/div/div/div/a/div/div[6]/div[1]"))
                    ).text.strip()

                    # 평점 문자열에서 숫자만 추출 (숫자와 점만 남기기)
                    rating = ''.join([c for c in rating if c.isdigit() or c == '.'])

                    # 문자열을 숫자(실수)로 변환
                    rating = float(rating) if rating else None
                except Exception as e:
                    print(f"   ❌ 새로운 XPath로 평점 추출 실패: {e}")

            if rating is not None:
                print(f"   ⭐ 숙소 평점: {rating}")
            else:
                print("   ❌ 숙소 평점을 찾을 수 없습니다.")

            # 데이터 저장
            data_list.append({
                "숙소 링크": link,
                "평점": rating,
                "이미지 파일": img_path if img_url else "N/A"
            })

            # 새 탭 닫기
            driver.close()

            # 다시 원래 탭으로 전환
            driver.switch_to.window(all_windows[0])

        except Exception as e:
            print(f"❌ {i}. 정보를 가져오는 데 실패했습니다: {e}")

# 크롤링을 시작하는 함수
def crawl_pages(start_page, end_page):
    global data_list

    for page in range(start_page, end_page + 1):
        print(f"\n🔹 {page} 페이지 크롤링 시작!")

        if page > 1:
            # 페이지 전환 (다음 페이지 버튼 클릭)
            next_page_button = driver.find_element(By.XPATH, f'//*[@id="site-content"]/div/div[3]/div/div/div/nav/div/a[{page}]')
            next_page_button.click()
            time.sleep(5)

        # 각 페이지에서 데이터 크롤링
        get_data_from_page()

    # 데이터 저장 (CSV 파일)
    df = pd.DataFrame(data_list)
    csv_path = "airbnb_data.csv"
    df.to_csv(csv_path, index=False, encoding="utf-8-sig")
    print(f"\n✅ 데이터 저장 완료: {csv_path}")

# 크롤링 시작
start_page = 1
end_page = 2  # 원하는 페이지 범위로 설정
crawl_pages(start_page, end_page)

# 드라이버 종료
driver.quit()


KeyboardInterrupt: 

In [ ]:
data = pd.read_csv('airbnb_data.csv')
data

,숙소 링크,평점,이미지 파일
0,https://www.airbnb.co.kr/rooms/539454505210849...,4.83,airbnb_images\숙소_1.jpg
1,https://www.airbnb.co.kr/rooms/47020322?search...,4.83,airbnb_images\숙소_2.jpg
2,https://www.airbnb.co.kr/rooms/34909224?search...,4.86,airbnb_images\숙소_3.jpg
3,https://www.airbnb.co.kr/rooms/975932091326226...,4.67,airbnb_images\숙소_4.jpg
4,https://www.airbnb.co.kr/rooms/119846594800169...,4.44,airbnb_images\숙소_5.jpg
5,https://www.airbnb.co.kr/rooms/996870015464652...,4.97,airbnb_images\숙소_6.jpg
6,https://www.airbnb.co.kr/rooms/124836023710986...,4.98,airbnb_images\숙소_7.jpg
7,https://www.airbnb.co.kr/rooms/123174153686375...,4.90,airbnb_images\숙소_8.jpg
8,https://www.airbnb.co.kr/rooms/135177682676099...,5.00,airbnb_images\숙소_9.jpg
9,https://www.airbnb.co.kr/rooms/114399466548469...,5.00,airbnb_images\숙소_10.jpg


In [ ]:
import time
import os
import requests
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from PIL import Image
from io import BytesIO

# 웹드라이버 실행
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# 크롤링할 URL 설정
url = "https://www.airbnb.co.kr/s/%EC%84%9C%EC%9A%B8/homes?refinement_paths%5B%5D=%2Fhomes"

# Airbnb 사이트 열기
driver.get(url)

# 페이지 로딩 대기
time.sleep(5)

print("✅ Airbnb 페이지가 성공적으로 열렸습니다.")

# 이미지 저장 폴더 생성
img_folder = "airbnb_images"
os.makedirs(img_folder, exist_ok=True)

# 데이터 저장 리스트
data_list = []

# 전역 이미지 인덱스
img_index = 1  

# 함수: 숙소 데이터를 가져오는 부분
def get_data_from_page():
    global data_list, img_index

    # 숙소 목록에서 링크 가져오기
    for i in range(1, 19):  # 상위 18개 숙소 확인
        try:
            # 숙소 링크 요소 찾기
            xpath = f"(//div[contains(@class, 'c4mnd7m')])[{i}]//a"
            link_element = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, xpath)))

            # 숙소 URL 가져오기
            link = link_element.get_attribute("href")
            print(f"🔹 {img_index}. 숙소 링크: {link}")

            # 새 탭 열기
            driver.execute_script(f"window.open('{link}', '_blank');")
            time.sleep(3)

            # 모든 열린 탭 가져오기
            all_windows = driver.window_handles
            driver.switch_to.window(all_windows[-1])  # 새로 열린 탭으로 이동

            # 첫 번째 이미지 요소 찾기
            img_url = None
            try:
                img_element = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, "//picture//img"))
                )
                img_url = img_element.get_attribute("src")

                if img_url:
                    print(f"   🖼️ 첫 번째 이미지 URL: {img_url}")

                    # 이미지 다운로드 및 저장
                    response = requests.get(img_url)
                    if response.status_code == 200:
                        img = Image.open(BytesIO(response.content))
                        img_path = os.path.join(img_folder, f"숙소_{img_index}.jpg")
                        img.save(img_path)
                        print(f"   ✅ 이미지 저장 완료: {img_path}")
                    else:
                        print(f"   ❌ 이미지 다운로드 실패")
                else:
                    print("   ❌ 이미지 URL을 찾을 수 없습니다.")

            except Exception as e:
                print(f"   ❌ 첫 번째 이미지를 가져오는 데 실패했습니다: {e}")

            # 숙소 평점(리뷰 점수) 가져오기
            rating = None
            try:
                # 기존 XPath로 평점 추출 시도
                rating = WebDriverWait(driver, 3).until(
                    EC.presence_of_element_located((By.XPATH, "//div[@class='r1lutz1s atm_c8_o7aogt atm_c8_l52nlx__oggzyc dir dir-ltr']"))
                ).text.strip()

                # 평점 문자열에서 숫자만 추출 (숫자와 점만 남기기)
                rating = ''.join([c for c in rating if c.isdigit() or c == '.'])

                # 문자열을 숫자(실수)로 변환
                rating = float(rating) if rating else None
            except Exception as e:
                print(f"   ❌ 기존 XPath로 평점 추출 실패: {e}")

            # 새로운 XPath로 평점 추출 시도
            if rating is None:
                try:
                    rating = WebDriverWait(driver, 3).until(
                        EC.presence_of_element_located((By.XPATH, "//*[@id='site-content']/div/div[1]/div[3]/div/div[1]/div/div[2]/div/div/div/a/div/div[6]/div[1]"))
                    ).text.strip()

                    # 평점 문자열에서 숫자만 추출 (숫자와 점만 남기기)
                    rating = ''.join([c for c in rating if c.isdigit() or c == '.'])

                    # 문자열을 숫자(실수)로 변환
                    rating = float(rating) if rating else None
                except Exception as e:
                    print(f"   ❌ 새로운 XPath로 평점 추출 실패: {e}")

            if rating is not None:
                print(f"   ⭐ 숙소 평점: {rating}")
            else:
                print("   ❌ 숙소 평점을 찾을 수 없습니다.")

            # 데이터 저장
            data_list.append({
                "숙소 링크": link,
                "평점": rating,
                "이미지 파일": img_path if img_url else "N/A"
            })

            # 새 탭 닫기
            driver.close()

            # 다시 원래 탭으로 전환
            driver.switch_to.window(all_windows[0])

            # 이미지 인덱스 증가
            img_index += 1  

        except Exception as e:
            print(f"❌ {i}. 정보를 가져오는 데 실패했습니다: {e}")

# 크롤링을 시작하는 함수
def crawl_pages(start_page, end_page):
    global data_list, img_index

    for page in range(start_page, end_page + 1):
        print(f"\n🔹 {page} 페이지 크롤링 시작!")

        if page > 1:
            # 페이지 전환 (다음 페이지 버튼 클릭)
            next_page_button = driver.find_element(By.XPATH, f'//*[@id="site-content"]/div/div[3]/div/div/div/nav/div/a[{page}]')
            next_page_button.click()
            time.sleep(5)

        # 각 페이지에서 데이터 크롤링
        get_data_from_page()

    # 데이터 저장 (CSV 파일)
    df = pd.DataFrame(data_list)
    csv_path = "airbnb_data.csv"
    df.to_csv(csv_path, index=False, encoding="utf-8-sig")
    print(f"\n✅ 데이터 저장 완료: {csv_path}")

# 크롤링 시작
start_page = 1
end_page = 3  # 원하는 페이지 범위로 설정
crawl_pages(start_page, end_page)

# 드라이버 종료
driver.quit()


✅ Airbnb 페이지가 성공적으로 열렸습니다.

🔹 1 페이지 크롤링 시작!
🔹 1. 숙소 링크: https://www.airbnb.co.kr/rooms/996870015464652023?search_mode=regular_search&adults=1&check_in=2025-03-24&check_out=2025-03-29&children=0&infants=0&pets=0&source_impression_id=p3_1742481452_P3J9b3Hl3BMSdux6&previous_page_section_name=1000&federated_search_id=81271504-0639-409a-9c90-18573eba6c33
   🖼️ 첫 번째 이미지 URL: https://a0.muscache.com/im/ml/photo_enhancement/pictures/hosting/Hosting-996870015464652023/original/14794a85-22bf-4645-8e3a-8e33f85aa7d1.jpeg?im_w=720
   ✅ 이미지 저장 완료: airbnb_images\숙소_1.jpg
   ❌ 기존 XPath로 평점 추출 실패: Message: 
Stacktrace:
	GetHandleVerifier [0x00C9B593+24387]
	(No symbol) [0x00C258F4]
	(No symbol) [0x00B00753]
	(No symbol) [0x00B48BA9]
	(No symbol) [0x00B48EFB]
	(No symbol) [0x00B919C2]
	(No symbol) [0x00B6D894]
	(No symbol) [0x00B8F138]
	(No symbol) [0x00B6D646]
	(No symbol) [0x00B3C59F]
	(No symbol) [0x00B3D8E4]
	GetHandleVerifier [0x00F9D873+3179043]
	GetHandleVerifier [0x00FB6CE9+3282585]
	GetHandleVe

In [ ]:
data = pd.read_csv('airbnb_data.csv')
data

,숙소 링크,평점,이미지 파일
0,https://www.airbnb.co.kr/rooms/996870015464652...,4.97,airbnb_images\숙소_1.jpg
1,https://www.airbnb.co.kr/rooms/134163283556991...,5.00,airbnb_images\숙소_2.jpg
2,https://www.airbnb.co.kr/rooms/683864721098941...,4.92,airbnb_images\숙소_3.jpg
3,https://www.airbnb.co.kr/rooms/868358169521821...,4.83,airbnb_images\숙소_4.jpg
4,https://www.airbnb.co.kr/rooms/137000936794274...,5.00,airbnb_images\숙소_5.jpg
5,https://www.airbnb.co.kr/rooms/121269209308295...,4.90,airbnb_images\숙소_6.jpg
6,https://www.airbnb.co.kr/rooms/115972553161535...,4.89,airbnb_images\숙소_7.jpg
7,https://www.airbnb.co.kr/rooms/52558539?search...,4.78,airbnb_images\숙소_8.jpg
8,https://www.airbnb.co.kr/rooms/54238311?search...,4.89,airbnb_images\숙소_9.jpg
9,https://www.airbnb.co.kr/rooms/134095495074182...,5.00,airbnb_images\숙소_10.jpg


In [ ]:
data = pd.read_csv('airbnb_data.csv')
data

,숙소 링크,평점,이미지 파일
0,https://www.airbnb.co.kr/rooms/539454505210849...,NaN,airbnb_images\숙소_1.jpg
1,https://www.airbnb.co.kr/rooms/131486600367044...,NaN,airbnb_images\숙소_2.jpg
2,https://www.airbnb.co.kr/rooms/34909224?search...,NaN,airbnb_images\숙소_3.jpg
3,https://www.airbnb.co.kr/rooms/47020322?search...,NaN,airbnb_images\숙소_4.jpg
4,https://www.airbnb.co.kr/rooms/975932091326226...,NaN,airbnb_images\숙소_5.jpg
5,https://www.airbnb.co.kr/rooms/119846594800169...,NaN,airbnb_images\숙소_6.jpg
6,https://www.airbnb.co.kr/rooms/117814071780884...,NaN,airbnb_images\숙소_7.jpg
7,https://www.airbnb.co.kr/rooms/996870015464652...,NaN,airbnb_images\숙소_8.jpg
8,https://www.airbnb.co.kr/rooms/124836023710986...,NaN,airbnb_images\숙소_9.jpg
9,https://www.airbnb.co.kr/rooms/123174153686375...,NaN,airbnb_images\숙소_10.jpg


In [ ]:
import time
import os
import requests
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from PIL import Image
from io import BytesIO
import concurrent.futures

# 웹드라이버 실행
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# 크롤링할 URL 설정
url = "https://www.airbnb.co.kr/s/%EC%84%9C%EC%9A%B8/homes?refinement_paths%5B%5D=%2Fhomes"

# Airbnb 사이트 열기
driver.get(url)

# 페이지 로딩 대기
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//*[contains(@class, 'c4mnd7m')]")))

print("✅ Airbnb 페이지가 성공적으로 열렸습니다.")

# 이미지 저장 폴더 생성
img_folder = "airbnb_images"
os.makedirs(img_folder, exist_ok=True)

# 데이터 저장 리스트
data_list = []
img_index = 1  

# 함수: 숙소 데이터를 가져오는 부분
def get_data_from_page():
    global data_list, img_index
    for i in range(1, 19):  # 상위 18개 숙소 확인
        try:
            xpath = f"(//div[contains(@class, 'c4mnd7m')])[{i}]//a"
            link_element = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, xpath)))
            link = link_element.get_attribute("href")
            print(f"🔹 {img_index}. 숙소 링크: {link}")
            
            # 이미지 URL을 가져오는 함수
            def fetch_image_url():
                driver.execute_script(f"window.open('{link}', '_blank');")
                time.sleep(2)
                all_windows = driver.window_handles
                driver.switch_to.window(all_windows[-1])  
                
                try:
                    img_element = WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.XPATH, "//picture//img"))
                    )
                    img_url = img_element.get_attribute("src")
                    return img_url
                except:
                    return None
            
            # 이미지 URL 비동기적으로 가져오기
            img_url = fetch_image_url()
            
            if img_url:
                print(f"   🖼️ 첫 번째 이미지 URL: {img_url}")
                # 이미지 다운로드 및 저장
                response = requests.get(img_url)
                if response.status_code == 200:
                    img = Image.open(BytesIO(response.content))
                    img_path = os.path.join(img_folder, f"숙소_{img_index}.jpg")
                    img.save(img_path)
                    print(f"   ✅ 이미지 저장 완료: {img_path}")
                else:
                    print(f"   ❌ 이미지 다운로드 실패")

            rating = None
            try:
                rating = WebDriverWait(driver, 3).until(
                    EC.presence_of_element_located((By.XPATH, "//div[@class='r1lutz1s atm_c8_o7aogt atm_c8_l52nlx__oggzyc dir dir-ltr']"))
                ).text.strip()
                rating = ''.join([c for c in rating if c.isdigit() or c == '.'])
                rating = float(rating) if rating else None
            except Exception as e:
                print(f"   ❌ 평점 추출 실패: {e}")
            
            if rating is None:
                try:
                    rating = WebDriverWait(driver, 3).until(
                        EC.presence_of_element_located((By.XPATH, "//*[@id='site-content']/div/div[1]/div[3]/div/div[1]/div/div[2]/div/div/div/a/div/div[6]/div[1]"))
                    ).text.strip()

                    # 평점 문자열에서 숫자만 추출 (숫자와 점만 남기기)
                    rating = ''.join([c for c in rating if c.isdigit() or c == '.'])

                    # 문자열을 숫자(실수)로 변환
                    rating = float(rating) if rating else None
                except Exception as e:
                    print(f"   ❌ 새로운 XPath로 평점 추출 실패: {e}")
            
            # 데이터 저장
            data_list.append({
                "숙소 링크": link,
                "평점": rating,
                "이미지 파일": img_path if img_url else "N/A"
            })

            driver.close()  # 새 탭 닫기
            driver.switch_to.window(driver.window_handles[0])  # 원래 탭으로 돌아가기

            img_index += 1  

        except Exception as e:
            print(f"❌ {i}. 정보를 가져오는 데 실패했습니다: {e}")

# 크롤링을 시작하는 함수
def crawl_pages(start_page, end_page):
    global data_list, img_index
    for page in range(start_page, end_page + 1):
        print(f"\n🔹 {page} 페이지 크롤링 시작!")
        if page > 1:
            next_page_button = driver.find_element(By.XPATH, f'//*[@id="site-content"]/div/div[3]/div/div/div/nav/div/a[{page}]')
            next_page_button.click()
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//*[contains(@class, 'c4mnd7m')]") ))
        
        get_data_from_page()


    # 데이터 저장 (CSV 파일)
    df = pd.DataFrame(data_list)
    csv_path = "airbnb_data.csv"
    df.to_csv(csv_path, index=False, encoding="utf-8-sig")
    print(f"\n✅ 데이터 저장 완료: {csv_path}")

# 크롤링 시작
start_page = 1
end_page = 100  # 원하는 페이지 범위로 설정
crawl_pages(start_page, end_page)

# 드라이버 종료
driver.quit()

## 보완할점 ddd


✅ Airbnb 페이지가 성공적으로 열렸습니다.

🔹 1 페이지 크롤링 시작!
🔹 1. 숙소 링크: https://www.airbnb.co.kr/rooms/1115276803608150224?search_mode=regular_search&adults=1&check_in=2025-04-20&check_out=2025-04-25&children=0&infants=0&pets=0&source_impression_id=p3_1742619854_P3nOMOyKCVz9wO0R&previous_page_section_name=1000&federated_search_id=9d69e449-40ee-40ee-8ccd-708cd2c88ce8
   🖼️ 첫 번째 이미지 URL: https://a0.muscache.com/im/pictures/hosting/Hosting-1115276803608150224/original/54853350-32c8-4e02-a8d7-f20a555ae6a9.jpeg?im_w=720
   ✅ 이미지 저장 완료: airbnb_images\숙소_1.jpg
   ❌ 평점 추출 실패: Message: 
Stacktrace:
	GetHandleVerifier [0x0110C7F3+24435]
	(No symbol) [0x01092074]
	(No symbol) [0x00F606E3]
	(No symbol) [0x00FA8B39]
	(No symbol) [0x00FA8E8B]
	(No symbol) [0x00FF1AC2]
	(No symbol) [0x00FCD804]
	(No symbol) [0x00FEF20A]
	(No symbol) [0x00FCD5B6]
	(No symbol) [0x00F9C54F]
	(No symbol) [0x00F9D894]
	GetHandleVerifier [0x014170A3+3213347]
	GetHandleVerifier [0x0142B0C9+3295305]
	GetHandleVerifier [0x0142558C+3271948]
	

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="site-content"]/div/div[3]/div/div/div/nav/div/a[6]"}
  (Session info: chrome=134.0.6998.118); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x0110C7F3+24435]
	(No symbol) [0x01092074]
	(No symbol) [0x00F606E3]
	(No symbol) [0x00FA8B39]
	(No symbol) [0x00FA8E8B]
	(No symbol) [0x00FF1AC2]
	(No symbol) [0x00FCD804]
	(No symbol) [0x00FEF20A]
	(No symbol) [0x00FCD5B6]
	(No symbol) [0x00F9C54F]
	(No symbol) [0x00F9D894]
	GetHandleVerifier [0x014170A3+3213347]
	GetHandleVerifier [0x0142B0C9+3295305]
	GetHandleVerifier [0x0142558C+3271948]
	GetHandleVerifier [0x011A7360+658144]
	(No symbol) [0x0109B27D]
	(No symbol) [0x01098208]
	(No symbol) [0x010983A9]
	(No symbol) [0x0108AAC0]
	BaseThreadInitThunk [0x76B25D49+25]
	RtlInitializeExceptionChain [0x779C